In [ ]:

import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 2. LOAD DATA
df = pd.read_csv("dataset/socal2.csv")

print("Dataset shape:", df.shape)
print(df.head())

X = df[['bed', 'bath', 'sqft']]
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# 3. SCALE TABULAR FEATURES

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

train_df = pd.DataFrame(X_train_scaled, columns=['bed','bath','sqft'])
train_df['price'] = y_train.values

test_df = pd.DataFrame(X_test_scaled, columns=['bed','bath','sqft'])
test_df['price'] = y_test.values

# 4. CUSTOM DATASET

class HouseDataset(Dataset):
    def __init__(self, df, image_dir):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        tabular = torch.tensor(
            [row['bed'], row['bath'], row['sqft']],
            dtype=torch.float32
        )
        
        price = torch.tensor(row['price'], dtype=torch.float32)
        
        img_path = os.path.join(self.image_dir, f"{idx}.jpg")
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        
        return image, tabular, price

# 5. DATALOADER
image_dir = "dataset/socal2/socal_pics"

train_dataset = HouseDataset(train_df, image_dir)
test_dataset  = HouseDataset(test_df, image_dir)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=16)


# 6. MULTIMODAL MODEL

class MultiModalModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Image branch using pretrained ResNet18
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Linear(self.cnn.fc.in_features, 128)
        
        # Tabular branch
        self.tabular_net = nn.Sequential(
            nn.Linear(3, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )
        
        # Combined branch
        self.combined = nn.Sequential(
            nn.Linear(128 + 32, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        
    def forward(self, image, tabular):
        img_feat = self.cnn(image)
        tab_feat = self.tabular_net(tabular)
        
        combined = torch.cat((img_feat, tab_feat), dim=1)
        output = self.combined(combined)
        
        return output.squeeze()


# 7. TRAINING SETUP

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = MultiModalModel().to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


# 8. TRAINING LOOP

EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    
    for images, tabs, targets in train_loader:
        images = images.to(device)
        tabs = tabs.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(images, tabs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss/len(train_loader)}")


# 9. EVALUATION

model.eval()
preds = []
actuals = []

with torch.no_grad():
    for images, tabs, targets in test_loader:
        images = images.to(device)
        tabs = tabs.to(device)
        
        outputs = model(images, tabs)
        
        preds.extend(outputs.cpu().numpy())
        actuals.extend(targets.numpy())

mae = mean_absolute_error(actuals, preds)
rmse = np.sqrt(mean_squared_error(actuals, preds))

print("MAE:", mae)
print("RMSE:", rmse)

Dataset shape: (15474, 8)
   image_id                 street             citi  n_citi  bed  bath  sqft  \
0         0  1317 Van Buren Avenue  Salton City, CA     317    3   2.0  1560   
1         1         124 C Street W      Brawley, CA      48    3   2.0   713   
2         2        2304 Clark Road     Imperial, CA     152    3   1.0   800   
3         3     755 Brawley Avenue      Brawley, CA      48    3   1.0  1082   
4         4  2207 R Carrillo Court     Calexico, CA      55    4   3.0  2547   

    price  
0  201900  
1  228500  
2  273950  
3  350000  
4  385100  
Using device: cpu


c:\Users\Hassan\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Hassan\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Hassan/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:49<00:00, 955kB/s] 


Epoch 1/10, Loss: 628597548849.6124
Epoch 2/10, Loss: 558662822994.0259
Epoch 3/10, Loss: 368955591111.1111
Epoch 4/10, Loss: 185286386407.52454
Epoch 5/10, Loss: 139449092645.0439
Epoch 6/10, Loss: 135309324809.26099
Epoch 7/10, Loss: 128577655278.80104
Epoch 8/10, Loss: 110509096192.6615
Epoch 9/10, Loss: 86909784709.62274
Epoch 10/10, Loss: 69974084963.8863
MAE: 331792.704069063
RMSE: 453889.40291520325
